<a href="https://colab.research.google.com/github/DonJM/FCLB-TFOD/blob/master/FCLB_TFOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install wget

#import neccessary python modules and libraries
import os
import shutil
from PIL import ImageEnhance
from PIL import Image
import uuid
import tarfile
import wget

# Adding Classes for Path Setup and Project Dataset

In [35]:
class Setup_Path: 

    LABELS = ["fresh", "rotten"]

    def getFiles(self):
      files = {
          "LABELMAP":os.path.join(self.getWorkspaces()['ANNOTATIONS_PATH'], "label_map.pbtxt"),
          'TFRECORD_SCRIPT':os.path.join(self.getWorkspaces()['PREPROCESSING_PATH'], 'generate_tfrecord.py'),
          "PRETRAINED_MODEL_NAME": 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
          "PRETRAINED_MODEL_URL": 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz', 
      }

      return files

    def getWorkspaces(self):
        paths = {
            "ADDONS_PATH":os.path.join("Tensorflow", "addons"),
            "LABEL_IMG_PATH":os.path.join("Tensorflow", "addons", "labelImg"),
            "TFOD_API_PATH":"Tensorflow",
            "WORKSPACE_PATH":os.path.join("Tensorflow", "workspace"),
            "TRAINING_DEMO_PATH":os.path.join("Tensorflow", "workspace", "training_demo"),
            "ANNOTATIONS_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "annotations"),
            "IMAGES_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "images"),
            "TRAINING_IMAGES_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "images", "train"),
            "TESTING_IMAGES_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "images", "test"),
            "PRETRAINED_MODELS_PATH":os.path.join("Tensorflow", "workspace", "training_demo", "pre-trained-models"),
            "PREPROCESSED_IMG":os.path.join("Tensorflow", "addons", "preprocessedImg"),
            "ORIGINAL_IMGS":os.path.join("Tensorflow", "addons", "originalImg"),   
            "PREPROCESSING_PATH":os.path.join("Tensorflow", "scripts", "preprocessing"),
            "PROTOS_PATH":os.path.join("Tensorflow", "Google-Protobuf"),
            'CHECKPOINT_PATH': os.path.join("Tensorflow", "workspace", "training_demo", 'my_ssd_mobnet'),
            'TFJS_PATH':os.path.join('Tensorflow', 'workspace', "training_demo",'models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
            'TFLITE_PATH':os.path.join('Tensorflow', 'workspace', "training_demo",'models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
        }

        return paths

    # def set_main_paths(self):
    #     for path in self.get_main_paths().values():
    #         if not os.path.exists(path):
    #             os.makedirs(path)

    # def get_image_paths(self):
    #     path = {
    #         "ORIGINAL_IMAGES_PATH" : ["workspace\datasets\original_images\Fresh_Ripe", "workspace\datasets\original_images\Fresh_Unripe", "workspace\datasets\original_images\Rotten_Ripe", "workspace\datasets\original_images\Rotten_Unripe"],
    #         "PREPROCESSED_IMAGES_PATH" : ["workspace\datasets\preprocessed_images\Fresh_Ripe", "workspace\datasets\preprocessed_images\Fresh_Unripe", "workspace\datasets\preprocessed_images\Rotten_Ripe", "workspace\datasets\preprocessed_images\Rotten_Unripe"],
    #     }

    #     return path

    # def set_image_paths(self):
    #     for path in self.get_image_paths().values():
    #         for label in path:
    #             if not os.path.exists(label):
    #                 os.makedirs(label)

    # def getFiles(self):
    #     files = {
    #         'PIPELINE_CONFIG':os.path.join(self.get_paths()['MODELS_PATH'], self.custom_name, 'pipeline.config'),
    #         'TF_RECORD_SCRIPT':os.path.join(self.get_paths()['TFRECORD_GENERATOR_PATH'], self.tfrecord_name),
    #         'LABEL_MAP':os.path.join(self.get_paths()['ANOTATIONS_PATH'], self.map_name),
    #     }
    #     return files

In [19]:
class FCLB_Datasets:
    #class' constructor
    def __init__(self, setup):
        self.setup = setup
    
    #acquiring uncategorized dataset
    def get_original_dataset(self, num1, num2, label):
        images_path = self.setup.getWorkspaces()['ORIGINAL_IMGS']
        path = os.path.join(images_path, label)

        files = ["{}".format(i) for i in os.listdir(path)[num1:num2]]

        return files

    def copy_image_file(self, num1, num2, srcPath, dstPath):

        for label in self.setup.LABELS:
            for file in self.get_original_dataset(num1, num2, label):
                dst = os.path.join(dstPath, label, file)
                src = os.path.join(srcPath, label, file)
                shutil.copyfile(src, dst)

    #I call it segment function, wala na koy mahunahunaan
    def segment(self):
        original_dir = self.setup.getWorkspaces()['ORIGINAL_IMGS']
        target_dir = self.setup.getWorkspaces()['PREPROCESSED_IMG']
        labels = self.setup.LABELS

        for label in labels:
            original = os.path.join(original_dir, label)
            target = os.path.join(target_dir, label)
            for img in os.listdir(original):
                self.segmentation(original, target, img, label)
 

    #this function used to segment an image file, it actually resize image, add contrast and lightness
    #and save it to the target directory
    #the purpose of uuid module is to give unique id or name to an image file
    def segmentation(self, base_dir, target_dir, fname, label):
        image = Image.open(os.path.join(base_dir, fname))
        size = (400,400)
        image.thumbnail(size)

        contrast = ImageEnhance.Contrast(image)
        contrast.enhance(1.8).save(os.path.join(target_dir, label + "-"+ '{}.jpg'.format(uuid.uuid1())))

        brightness = ImageEnhance.Brightness(image)
        brightness.enhance(1.2).save(os.path.join(target_dir, label +"-"+ '{}.jpg'.format(uuid.uuid1())))

    #compressed file for github upload
    def compressed(self):
        path = self.getDir('PREPROCESSED_IMAGES_PATH')
        sub_paths = self.getSubDirectories(path)
        ARCHIVE_PATH = os.path.join(path, 'preprocessed_images.tar.gz')
        tar = tarfile.open(ARCHIVE_PATH, "w:gz")
        for sub_path in sub_paths:
            dir = os.path.join(path, sub_path)
            tar.add(dir)
        tar.close()

# Preparing Workspace

Creating a workspace directory

In [36]:
setup = Setup_Path()
workspaces = setup.getWorkspaces()
for workspace in workspaces.values():
  if not os.path.exists(workspace):
    os.makedirs(workspace)

In [ ]:
!unzip /content/train.zip
shutil.move('/content/fresh', setup.getWorkspaces()['ORIGINAL_IMGS'])
shutil.move('/content/rotten', setup.getWorkspaces()['ORIGINAL_IMGS'])

Archive:  /content/train.zip
replace fresh/Fresh.526.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace fresh/Fresh.534.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace fresh/Fresh.534.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: fresh/Fresh.534.jpg     
replace fresh/Fresh.718.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: fresh/Fresh.718.jpg     
replace fresh/Fresh.729.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: fresh/Fresh.729.jpg     
replace fresh/Fresh.743.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: fresh/Fresh.743.jpg     
replace fresh/Fresh.799.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: fresh/Fresh.799.jpg     
replace rotten/Rotten.120.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: rotten/Rotten.120.jpg   
replace rotten/Rotten.1538.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: rotten/Rotten.1538.jpg  
replace rotten/Rotten.1553.jpg? [y]es, [n]o, [A]ll

'Tensorflow/addons/originalImg/rotten'

Create label directory for original images and preprocessed images

In [ ]:
for label in setup.LABELS:
  preprocessedImg_labelPath = os.path.join(setup.getWorkspaces()['PREPROCESSED_IMG'], label)
  if not os.path.exists(preprocessedImg_labelPath):
    os.makedirs(preprocessedImg_labelPath)

segment images and copy to preprocessed directory

In [ ]:
dataset = FCLB_Datasets(setup)
dataset.segment()

In [ ]:
!cd /content/Tensorflow && zip {setup.getWorkspaces()['PREPROCESSED_IMG']}


zip error: Nothing to do! (Tensorflow/addons/preprocessedImg.zip)


# Image Labelling

In [ ]:
labelImg = setup.getWorkspaces()['LABEL_IMG_PATH']
!cd {labelImg} && git clone https://github.com/tzutalin/labelImg.git

Cloning into 'labelImg'...
remote: Enumerating objects: 1846, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 1846 (delta 37), reused 55 (delta 18), pack-reused 1759
Receiving objects: 100% (1846/1846), 232.81 MiB | 41.39 MiB/s, done.
Resolving deltas: 100% (1084/1084), done.


In [ ]:
labelImg = os.path.join(setup.getWorkspaces()['LABEL_IMG_PATH'], "labelImg")
!cd {labelImg} && pip install pyqt5
!cd {labelImg} && pyrcc5 -o libs/resources.py resources.qrc

# Partition Dataset

In [ ]:
url = "https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/d0e545609c5f7f49f39abc7b6a38cec3/partition_dataset.py"
src = wget.download(url)
dst = setup.getWorkspaces()['PREPROCESSING_PATH']
shutil.move(src, dst)

'Tensorflow/scripts/preprocessing/partition_dataset.py'

In [ ]:
images_path = setup.getWorkspaces()['IMAGES_PATH']
!cd {dst} python partition_dataset.py -x -i {images_path} -r 0.1

/bin/bash: line 0: cd: too many arguments


# Download Tensorflow Object Detection API

In [21]:
tfod_api = setup.getWorkspaces()['TFOD_API_PATH']
!cd {tfod_api} && git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 61140, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 61140 (delta 14), reused 22 (delta 8), pack-reused 61110
Receiving objects: 100% (61140/61140), 574.01 MiB | 32.48 MiB/s, done.
Resolving deltas: 100% (42606/42606), done.


# Install Object Detection

In [22]:
research_path = os.path.join(setup.getWorkspaces()['TFOD_API_PATH'], 'models', 'research')
!cd {research_path} && cp object_detection/packages/tf2/setup.py setup.py

In [23]:
url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
src = wget.download(url)
protos_path = setup.getWorkspaces()['PROTOS_PATH']
shutil.move(src, protos_path)

'Tensorflow/Google-Protobuf/protoc-3.15.6-win64.zip'

In [24]:
!apt-get install protobuf-compiler
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Processing /content/Tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.8 MB 8.1 MB/s 
     |████████████████████████████████| 352 kB 44.2 MB/s 
     |████████████████████████████████| 1.8 MB 20.1 MB/s 
     |███████████████

In [25]:
# !cd {research_path} && python object_detection/builders/model_builder_tf2_test.py
VERIFICATION_SCRIPT = os.path.join(research_path, 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2021-09-02 05:33:53.557626: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-02 05:33:53.557715: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dcfb9ab4b5a6): /proc/driver/nvidia/version does not exist
Running tests under Python 3.7.11: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0902 05:33:53.948330 140467033573248 model_builder.py:1091] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.84s
I0902 05:33:54.403601 140467033573248 test_util.py:2189] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.84s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorf

In [26]:
!pip list

Package                       Version
----------------------------- --------------
absl-py                       0.12.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.1.0
apache-beam                   2.32.0
appdirs                       1.4.4
argcomplete                   1.12.3
argon2-cffi                   21.1.0
arviz                         0.11.2
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.2.0
audioread                     2.1.9
autograd                      1.3
avro-python3                  1.9.2.1
Babel                         2.9.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        4.0.0
blis                          0.4.1
bokeh                         2.3.3
Bottleneck                    1.3.2
branca   

In [27]:
import object_detection

# Create Label Map

In [29]:
labels = [{'name':'fresh', 'id':1}, {'name':'rotten', 'id':2}]

with open(setup.getFiles()['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# Create Tensorflow Records

In [30]:
url = 'https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py'
src = wget.download(url)
dst = setup.getWorkspaces()['PREPROCESSING_PATH']
shutil.move(src, dst)

'Tensorflow/scripts/preprocessing/generate_tfrecord.py'

In [31]:
# Create train data:
!python {setup.getFiles()['TFRECORD_SCRIPT']} -x {setup.getWorkspaces()['TRAINING_IMAGES_PATH']} -l {setup.getFiles()['LABELMAP']} -o {os.path.join(setup.getWorkspaces()['ANNOTATIONS_PATH'], 'train.record')}

# Create test data:
!python {setup.getFiles()['TFRECORD_SCRIPT']} -x {setup.getWorkspaces()['TESTING_IMAGES_PATH']} -l {setup.getFiles()['LABELMAP']} -o {os.path.join(setup.getWorkspaces()['ANNOTATIONS_PATH'], 'train.record')}

2021-09-02 05:36:37.642664: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecord file: Tensorflow/workspace/training_demo/annotations/train.record
2021-09-02 05:36:40.594050: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecord file: Tensorflow/workspace/training_demo/annotations/train.record


# Download Pretrained Model

In [45]:
# pretrained_mode_file = wget.download(setup.getFiles()['PRETRAINED_MODEL_URL'])
# shutil.move(pretrained_mode_file, setup.getWorkspaces()['PRETRAINED_MODELS_PATH'])
!cd {setup.getWorkspaces()['PRETRAINED_MODELS_PATH']} && tar -zxvf {pretrained_mode_file}

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index


# Make yourown copy of pre-trained model

In [46]:
!cp {os.path.join(setup.getWorkspaces()['PRETRAINED_MODELS_PATH'], 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8', 'pipeline.config')} {os.path.join(setup.getWorkspaces()['CHECKPOINT_PATH'])}

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

ModuleNotFoundError: ignored